In [1]:
from EMsolver.solver import EMsolver
import numpy as np
import math
import os
import cupy
from numba import cuda
import ray
from EMsolver.region_distance import signal_indicator
import matplotlib.pyplot as plt
from timeit import default_timer as timer


# import ray
ray.shutdown()
ray.init(_temp_dir='/data/ray')

2024-05-17 17:03:53,822	INFO worker.py:1622 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8268 


Python version:,3.7.13
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8268


In [2]:
def areal_position(i, d, left_boundary):
    return i*d + d/2 + left_boundary

# generate sinusodial sources of rho and J
def generate_rho_J(frequency,time, dx, dy, dz, x_left_boundary, y_left_boundary, z_left_boundary, \
                   x_grid_size, y_grid_size, z_grid_size, total_spatial_grid_size):    
    rho, Jx, Jy, Jz = (np.empty(total_spatial_grid_size, dtype=np.float32) for _ in range(4))
    for i_spatial in range(total_spatial_grid_size):
        iz = i_spatial%z_grid_size
        iz_rest = i_spatial//z_grid_size
        iy = iz_rest%y_grid_size
        iy_rest = iz_rest//y_grid_size
        ix = iy_rest%x_grid_size
        x, y, z = areal_position(ix, dx, x_left_boundary), areal_position(iy, dy, y_left_boundary), areal_position(iz, dz, z_left_boundary)
        if frequency ==0:
            Jx[i_spatial] = 5 * math.cos(2 * z) * math.sin(3 * x + y) * math.sin(4 * time)
            Jy[i_spatial] = 5 * math.sin(2 * z) * math.sin(3 * x + y) * math.cos(4 * time)
            Jz[i_spatial] = 5 * math.cos(2 * z) * math.sin(3 * x + y) * math.sin(4 * time)
            rho[i_spatial] = (5/8) * (2 * math.cos(4 * time - 3 * x - y - 2 * z) + math.cos(4 * time - 3 * x - y + 2 * z) + 3 * math.cos(4 * time + 3 * x + y + 2 * z))                           
    return Jx, Jy, Jz, rho

In [3]:
number_grids = 20
# the regions are seperated as the source region and the observation region
x_grid_size_o, y_grid_size_o, z_grid_size_o = number_grids,number_grids,number_grids
x_grid_size_s, y_grid_size_s, z_grid_size_s = number_grids,number_grids,number_grids

# the infinitesimals of the regions
# here the source region and observational region are overlap
dx_o, dy_o, dz_o, x_left_boundary_o, y_left_boundary_o, z_left_boundary_o = \
                       6/x_grid_size_o, 6/y_grid_size_o, 6/z_grid_size_o, -3, -3, -3
dx_s, dy_s, dz_s, x_left_boundary_s, y_left_boundary_s, z_left_boundary_s = \
                       6/x_grid_size_s, 6/y_grid_size_s, 6/z_grid_size_s, -3, -3, -3
dt = 0.005
# define the length of the sources
# rho_GPU and Jx_GPU are of shape [len_time_snapshots, total_grid_size]
len_time_snapshots = 10000

In [4]:
i_GPU = '3'


# load the remote server and set up the constant sources of \rho and J
f = EMsolver.remote(len_time_snapshots, i_GPU, \
                    x_grid_size_o, y_grid_size_o, z_grid_size_o, \
                    x_grid_size_s, y_grid_size_s, z_grid_size_s, \
                    dx_o, dy_o, dz_o, x_left_boundary_o, y_left_boundary_o, z_left_boundary_o, \
                    dx_s, dy_s, dz_s, x_left_boundary_s, y_left_boundary_s, z_left_boundary_s, \
                    dt)
       
# toy model of constant sources
rho= np.ones(x_grid_size_s*y_grid_size_s*z_grid_size_s, dtype=np.float32)
Jx, Jy, Jz = (np.ones(x_grid_size_s*y_grid_size_s*z_grid_size_s, dtype=np.float32) for _ in range(3))       


# This is for saving zero E and B, can be neglected if not used.
Ex, Ey, Ez, Bx, By, Bz = (np.zeros(x_grid_size_s*y_grid_size_s*z_grid_size_s, dtype=np.float32) for _ in range(6))

# save the results in these lists
Ex_list, Ey_list, Ez_list, Bx_list, By_list, Bz_list = (np.zeros(len_time_snapshots*x_grid_size_s*y_grid_size_s*z_grid_size_s, dtype=np.float32) for _ in range(6))
Jx_list, Jy_list, Jz_list, rho_list = (np.zeros(len_time_snapshots*x_grid_size_s*y_grid_size_s*z_grid_size_s, dtype=np.float32) for _ in range(4))    
zero_list = np.zeros(x_grid_size_s*y_grid_size_s*z_grid_size_s)
total_list = np.zeros((10,x_grid_size_s*y_grid_size_s*z_grid_size_s*len_time_snapshots), dtype=np.float32)

In [5]:

print("start")
# make sure if the signal has been transmitted to the observaional region
retarded_time = signal_indicator(dx_o, dy_o, dz_o, x_left_boundary_o, y_left_boundary_o, z_left_boundary_o, \
                                 dx_s, dy_s, dz_s, x_left_boundary_s, y_left_boundary_s, z_left_boundary_s, \
                                 x_grid_size_o, y_grid_size_o, z_grid_size_o, \
                                 x_grid_size_s, y_grid_size_s, z_grid_size_s)
for frequency in range(1):
    
    rho= np.ones(x_grid_size_s*y_grid_size_s*z_grid_size_s, dtype=np.float32)
    Jx, Jy, Jz = (np.ones(x_grid_size_s*y_grid_size_s*z_grid_size_s, dtype=np.float32) for _ in range(3))       


    # This is for saving zero E and B, can be neglected if not used.
    Ex, Ey, Ez, Bx, By, Bz = (np.zeros(x_grid_size_s*y_grid_size_s*z_grid_size_s, dtype=np.float32) for _ in range(6))

    # save the results in these lists
    Ex_list, Ey_list, Ez_list, Bx_list, By_list, Bz_list = (np.zeros(len_time_snapshots*x_grid_size_s*y_grid_size_s*z_grid_size_s, dtype=np.float32) for _ in range(6))
    Jx_list, Jy_list, Jz_list, rho_list = (np.zeros(len_time_snapshots*x_grid_size_s*y_grid_size_s*z_grid_size_s, dtype=np.float32) for _ in range(4))    
    zero_list = np.zeros(x_grid_size_s*y_grid_size_s*z_grid_size_s)
    total_list = np.zeros((10,x_grid_size_s*y_grid_size_s*z_grid_size_s*len_time_snapshots), dtype=np.float32)
    start = timer()
    for time in range(1):

        
        # updata new rho and J 
        Jx, Jy, Jz, rho = generate_rho_J(frequency,time*dt, dx_s, dy_s, dz_s, x_left_boundary_s, y_left_boundary_s, z_left_boundary_s, \
                                         x_grid_size_s, y_grid_size_s, z_grid_size_s, x_grid_size_s*y_grid_size_s*z_grid_size_s)

        f.update_rho_J.remote(rho, Jx, Jy, Jz)
        
        if time%1000 ==0:
            print(frequency,time)
        if time*dt < retarded_time:
            Jx_list[time*len(Jx):(time+1)*len(Jx)] = zero_list
            Jy_list[time*len(Jy):(time+1)*len(Jy)] = zero_list
            Jz_list[time*len(Jz):(time+1)*len(Jz)] = zero_list
            rho_list[time*len(rho):(time+1)*len(rho)] = zero_list
            Ex_list[time*len(Ex):(time+1)*len(Ex)] = zero_list
            Ey_list[time*len(Ey):(time+1)*len(Ey)] = zero_list
            Ez_list[time*len(Ez):(time+1)*len(Ez)] = zero_list
            Bx_list[time*len(Bx):(time+1)*len(Bx)] = zero_list
            By_list[time*len(By):(time+1)*len(By)] = zero_list
            Bz_list[time*len(Bz):(time+1)*len(Bz)] = zero_list

        else:
            Ex, Ey, Ez, Bx, By, Bz = ray.get(f.Jefimenko_solver.remote())

            Jx_list[time*len(Jx):(time+1)*len(Jx)] = Jx
            Jy_list[time*len(Jy):(time+1)*len(Jy)] = Jy
            Jz_list[time*len(Jz):(time+1)*len(Jz)] = Jz
            rho_list[time*len(rho):(time+1)*len(rho)] = rho
            Ex_list[time*len(Ex):(time+1)*len(Ex)] = Ex
            Ey_list[time*len(Ey):(time+1)*len(Ey)] = Ey
            Ez_list[time*len(Ez):(time+1)*len(Ez)] = Ez
            Bx_list[time*len(Bx):(time+1)*len(Bx)] = Bx
            By_list[time*len(By):(time+1)*len(By)] = By
            Bz_list[time*len(Bz):(time+1)*len(Bz)] = Bz

    total_list[0] =  Jx_list
    total_list[1] =  Jy_list
    total_list[2] =  Jz_list
    total_list[3] =  rho_list
    total_list[4] =  Ex_list
    total_list[5] =  Ey_list
    total_list[6] =  Ez_list
    total_list[7] =  Bx_list
    total_list[8] =  By_list
    total_list[9] =  Bz_list
    end = timer()
    np.save("/data/zhangjunjie/Data/Test_diff_frequence/Data_"+str(500+frequency),total_list)
    


print('evaluation time(hours): ',(end-start)/3600) 

start
0 0
evaluation time(hours):  0.022302033637242858


In [ ]:
start1 = timer()
x_grid_size_o, y_grid_size_o, z_grid_size_o = number_grids,number_grids,number_grids
x_grid_size_s, y_grid_size_s, z_grid_size_s = number_grids,number_grids,number_grids

# the infinitesimals of the regions
# here the source region and observational region are overlap
dx_o, dy_o, dz_o, x_left_boundary_o, y_left_boundary_o, z_left_boundary_o = \
                       6/x_grid_size_o, 6/y_grid_size_o, 6/z_grid_size_o, -3, -3, -3
dx_s, dy_s, dz_s, x_left_boundary_s, y_left_boundary_s, z_left_boundary_s = \
                       6/x_grid_size_s, 6/y_grid_size_s, 6/z_grid_size_s, -3, -3, -3
dt = 0.005

ix = np.zeros((number_grids,number_grids,number_grids))
iy = np.zeros((number_grids,number_grids,number_grids))
iz = np.zeros((number_grids,number_grids,number_grids))
dt_list = np.zeros((number_grids,number_grids,number_grids))
for i in range(number_grids):
    for j in range(number_grids):
        for k in range(number_grids):
            ix[i][j][k] = -2.85+(dx_s*i)
            iy[i][j][k] = -2.85+(dy_s*i)
            iz[i][j][k] = -2.85+(dz_s*i)
            dt_list[i][j][k] = dt * 5
end1  =  timer()

In [ ]:
start2 = timer()
number_g = number_grids*number_grids*number_grids
for s in range(1):
    
    total_list = np.load('/data/zhangjunjie/Data/Test_diff_frequence/Data_'+str(500+s)+'.npy',allow_pickle=True) 
    
    data_rho_J = np.zeros((1920,number_grids,number_grids,number_grids,4), dtype=np.float32)
    data_EM =  np.zeros((1920,number_grids,number_grids,number_grids,6), dtype=np.float32)
    label = np.zeros((1920,number_grids,number_grids,number_grids,6), dtype=np.float32)
    
    data_rho_J_middle = np.zeros((4,number_grids,number_grids,number_grids), dtype=np.float32)
    data_EM_middle =  np.zeros((6,number_grids,number_grids,number_grids), dtype=np.float32)
    label_middle = np.zeros((6,number_grids,number_grids,number_grids), dtype=np.float32)
    for q in range(1920):       
        data_rho_J_middle[0] = total_list[3][(q*5)*number_g:(q*5+1)*number_g].reshape([number_grids,number_grids,number_grids])
        data_rho_J_middle[1] = total_list[0][(q*5)*number_g:(q*5+1)*number_g].reshape([number_grids,number_grids,number_grids])
        data_rho_J_middle[2] = total_list[1][(q*5)*number_g:(q*5+1)*number_g].reshape([number_grids,number_grids,number_grids])
        data_rho_J_middle[3] = total_list[2][(q*5)*number_g:(q*5+1)*number_g].reshape([number_grids,number_grids,number_grids])



        data_EM_middle[0] = total_list[4][(q*5)*number_g:(q*5+1)*number_g].reshape([number_grids,number_grids,number_grids])
        data_EM_middle[1] = total_list[5][(q*5)*number_g:(q*5+1)*number_g].reshape([number_grids,number_grids,number_grids])
        data_EM_middle[2] = total_list[6][(q*5)*number_g:(q*5+1)*number_g].reshape([number_grids,number_grids,number_grids])
        data_EM_middle[3] = total_list[7][(q*5)*number_g:(q*5+1)*number_g].reshape([number_grids,number_grids,number_grids])
        data_EM_middle[4] = total_list[8][(q*5)*number_g:(q*5+1)*number_g].reshape([number_grids,number_grids,number_grids])
        data_EM_middle[5] = total_list[9][(q*5)*number_g:(q*5+1)*number_g].reshape([number_grids,number_grids,number_grids])


        label_middle[0] = total_list[4][((q+1)*5+1)*number_g:((q+1)*5+2)*number_g].reshape([number_grids,number_grids,number_grids])
        label_middle[1] = total_list[5][((q+1)*5+1)*number_g:((q+1)*5+2)*number_g].reshape([number_grids,number_grids,number_grids])
        label_middle[2] = total_list[6][((q+1)*5+1)*number_g:((q+1)*5+2)*number_g].reshape([number_grids,number_grids,number_grids])
        label_middle[3] = total_list[7][((q+1)*5+1)*number_g:((q+1)*5+2)*number_g].reshape([number_grids,number_grids,number_grids])
        label_middle[4] = total_list[8][((q+1)*5+1)*number_g:((q+1)*5+2)*number_g].reshape([number_grids,number_grids,number_grids])
        label_middle[5] = total_list[9][((q+1)*5+1)*number_g:((q+1)*5+2)*number_g].reshape([number_grids,number_grids,number_grids])
        
        data_rho_J[q] = np.transpose(data_rho_J_middle,(1,2,3,0))
        data_EM[q] = np.transpose(data_EM_middle,(1,2,3,0))
        label[q] = np.transpose(label_middle,(1,2,3,0))
#     np.save("/data/zhangjunjie/Data/Test_diff_frequence/data_rho_J-"+str(500+s),data_rho_J)
#     np.save("/data/zhangjunjie/Data/Test_diff_frequence/data_EM-"+str(500+s),data_EM)
#     np.save("/data/zhangjunjie/Data/Test_diff_frequence/label-"+str(500+s),label)
    print(q)
end2 = timer()
print('evaluation time(hours): ',(end-start)/3600) 

In [ ]:
start3 = timer()
for j in range(1):
#     inputs1 = np.load('/data/zhangjunjie/Data/Test_diff_frequence/data_rho_J-'+str(500+j)+'.npy',allow_pickle=True)
#     inputs2 = np.load('/data/zhangjunjie/Data/Test_diff_frequence/data_EM-'+str(500+j)+'.npy',allow_pickle=True)
#     labels = np.load('/data/zhangjunjie/Data/Test_diff_frequence/label-'+str(500+j)+'.npy',allow_pickle=True)
    
    inputs1 = data_rho_J
    inputs2 = data_EM
    labels = label
    input_time_rho = np.zeros((1920,number_grids,number_grids,number_grids,4,10))
    input_time_EM = np.zeros((1920,number_grids,number_grids,number_grids,6,10))
    label =np.zeros((1920,number_grids,number_grids,number_grids,6))
    for i in range(10,1920):
        input_time_rho[i]  = inputs1[i-10:i].transpose((1,2,3,4,0))
        input_time_EM[i] = inputs2[i-10:i].transpose((1,2,3,4,0))
        label[i] = labels[i-1]
    print(j)
    np.save("/data/zhangjunjie/Data/Test_diff_frequence/10_data_rho_J-"+str(500+j),input_time_rho)
    np.save("/data/zhangjunjie/Data/Test_diff_frequence/10_data_EM-"+str(500+j),input_time_EM)
    np.save("/data/zhangjunjie/Data/Test_diff_frequence/10_label-"+str(500+j),label)
end3 = timer()

In [ ]:
time_total = (end-start)/3600 + (end1-start1)/3600 + (end2-start2)/3600 + (end3-start3)/3600
print(time_total)